In [2]:
import joblib
import os

# 1. Define the path and ensure the folder exists
model_path = os.path.join('..', 'models')
os.makedirs(model_path, exist_ok=True)

# 2. Save the Baseline Model (scikit-learn)
baseline_file = os.path.join(model_path, 'baseline_heart_model.pkl')
joblib.dump(baseline_model, baseline_file)

# 3. Save the Deep Learning Model (Keras/TensorFlow)
# Saving in .keras format is the modern standard for TensorFlow 2.x+
nn_file = os.path.join(model_path, 'heart_disease_nn_model.keras')
model.save(nn_file)

print(f"✅ Success! Models saved in: {os.path.abspath(model_path)}")

NameError: name 'baseline_model' is not defined